In [1]:
%%bash
echo "deb [ arch=amd64 ] https://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
apt-get update
apt-get install python3-graph-tool

Executing: /tmp/apt-key-gpghome.cIISoIOHfx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,073 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://downloads.skewed.de/apt b

gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [2]:
import pandas as pd
edges = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_30last/edges.parquet')
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_30last/edges_test.parquet')

In [ ]:
#splitted
import pandas as pd
edges = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_30last/edges.parquet')
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_30last/edges_test.parquet')

In [3]:
JOIN = True

In [ ]:
JOIN = False

In [4]:
#Params
TEST_CLICK_WEIGHT = 3
TEST_CART_WEIGHT = 6
TEST_ORDER_WEIGHT_L= 6
TEST_ORDER_WEIGHT_S = 2


MIN_TOPK_COUSER = 500
COFITNESS_THRES = 0.01


ALPHA = 0.5
BETA = 0.5


USE_PAGERANK = False
USE_LOG = False

# Create graph

In [5]:
from graph_tool import Graph
ug = Graph(directed=False)

In [6]:
edges.iloc[:, 1] = edges.iloc[:,1]  + 100000000
edges.shape

(123300115, 7)

In [7]:
edges

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')"
0,0,100006851,2022018,2022018,1,0,0
1,0,100007563,2031353,2031315,2,0,0
2,0,100010268,1853151,1853151,1,0,0
3,0,100016246,62919,62919,1,0,0
4,0,100030373,1798887,1798887,1,0,0
...,...,...,...,...,...,...,...
123300110,12899776,100548599,2419172,2419172,1,0,0
123300111,12899776,101737908,2419187,2419187,1,0,0
123300112,12899777,100384045,2419186,2419176,2,0,0
123300113,12899778,100032070,2419194,2419194,1,0,0


In [8]:
edges_test.iloc[:, 1]= edges_test.iloc[:, 1] + 100000000
edges_test.shape

(5006557, 7)

In [9]:
edges_test

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')"
0,12899779,100059625,2419200,2419200,1,0,0
1,12899780,100582732,2419258,2419258,1,0,0
2,12899780,100736515,2419336,2419336,1,0,0
3,12899780,100973453,2419309,2419309,1,0,0
4,12899780,101142000,2419355,2419200,2,0,0
...,...,...,...,...,...,...,...
5006552,14571577,101141710,3023974,3023974,1,0,0
5006553,14571578,100519105,3023975,3023975,1,0,0
5006554,14571579,100739876,3023975,3023975,1,0,0
5006555,14571580,100202353,3023981,3023981,1,0,0


In [10]:
import numpy as np
edges_val = edges.values

In [11]:
import numpy as np
edges_test_val = edges_test.values

In [12]:
all_edges = np.vstack([edges_val, edges_test_val])

In [13]:
del edges_val, edges_test_val, edges, edges_test

In [14]:
all_edges.shape

(128306672, 7)

In [15]:
PR_CLICK_WEIGHT = 1
PR_CART_WEIGHT = 3
PR_ORDER_WEIGHT = 6
weight = all_edges[:, -3] * PR_CLICK_WEIGHT + all_edges[:, -2] * PR_CART_WEIGHT + all_edges[:, -1] * PR_ORDER_WEIGHT 

In [16]:
weight = weight.reshape(-1, 1)

In [17]:
all_edges = np.hstack([all_edges, weight])
all_edges.shape

(128306672, 8)

In [18]:
del weight

# Wait for delete

In [19]:
elts = ug.new_ep('unsigned long')
efts = ug.new_ep('unsigned long')
eclick = ug.new_ep('unsigned int')
ecart = ug.new_ep('unsigned int')
eorder = ug.new_ep('unsigned int')
# eweight = ug.new_ep('unsigned int')


In [20]:
# vmap = ug.add_edge_list(all_edges, eprops=[elts, efts, eclick, ecart, eorder, eweight])
vmap = ug.add_edge_list(all_edges, eprops=[elts, efts, eclick, ecart, eorder])

In [21]:
ug.num_edges()

128306672

In [22]:
del all_edges

# Pagerank

In [ ]:
#get pagerank
import graph_tool.centrality as gt
pr = gt.pagerank(ug, weight = eweight)

In [ ]:
#get log_pagerank
from graph_tool import map_property_values
log_pr = ug.new_vertex_property("double")
map_property_values(pr, log_pr,
                       lambda x: 10 + np.log10(x))

# Load test

In [23]:
ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
DATA_FOLDER = ROOT + '/data'



In [24]:
# ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
# DATA_FOLDER = ROOT + '/splitted_data'



In [25]:
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')

In [26]:
session_list = np.sort(test_df['session'].unique()).tolist()


In [27]:
# session_list = session_list[:len(session_list) // 100]

In [28]:
del test_df

# Find neighbour strategy

In [29]:
def get_test_items(user, beta, action):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  edges = ug.get_all_edges(user, eprops = [elts, eclick, ecart, eorder])
  
  args = {
      'test_click_weight': TEST_CLICK_WEIGHT,
      'test_cart_weight': TEST_CART_WEIGHT,
      'test_order_weight_l': TEST_ORDER_WEIGHT_L,
      'test_order_weight_s': TEST_ORDER_WEIGHT_S,
  }
  

  if edges.shape[0] < 20:
    
    return _get_test_items(edges, beta, **args)

  else:

    #CLICKS
    
    if action == 0:
      args['num_count_order'] = 10 #TUNED
      args['test_click_count'] = 1 #TUNED
      args['test_cart_count'] = 1 #TUNED
      args['test_order_count'] = 0 #TUNED

    #CARTS
    if action == 1:
      args['num_count_order'] = 10 #TUNED
      args['test_click_count'] = 1 #TUNED
      args['test_cart_count'] = 1 #TUNED
      args['test_order_count'] = 0 #TUNED
    
    #ORDERS
    if action == 2:
      args['num_count_order'] = 15 #TUNED
      args['test_click_count'] = 0 #TUNED
      args['test_cart_count'] = 3 #TUNED
      args['test_order_count'] = 1 #TUNED


    #add random 0 to have the same output number
    return _get_test_items_when_test_many(edges, beta, **args), 0



def get_user_neighbours(item):
  edges = ug.get_all_edges(item, eprops = [elts, efts, eclick, ecart, eorder])
  return _get_user_neighbours(edges)

def get_item_neighbours(user, lts, fts, prev_click, prev_cart, prev_order, alpha, use_pagerank, use_log, action):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order

  edges = ug.get_all_edges(user, eprops = [elts, efts, eclick, ecart, eorder])
  pagerank = ug.get_all_neighbors(user, vprops = [log_pr if use_log else pr])[:, 1] if use_pagerank else 1

  if action == 0:
    args = {
      'click_weight': 6, #TUNED
      'cart_weight' : 3,  #TUNED
      'order_weight': 1, #TUNED
      'day' : 8 * 3600,   #TUNED  
    }
  
  if action == 1:
    args = {
      'click_weight': 3, #TUNED
      'cart_weight' : 6,  #TUNED
      'order_weight': 1, #TUNED
      'day' : 8 * 3600,    #TUNED 
    }
  
  if action == 2:
    args = {
      'click_weight': 3, #TUNED
      'cart_weight' : 10,  #TUNED
      'order_weight': 1, #TUNED
      'day' : 8 * 3600,    #TUNED 
    }

  
  
  
  return _get_item_neighbours(edges, lts, fts, prev_click = prev_click, prev_cart = prev_cart, prev_order = prev_order
                              , pagerank = pagerank, alpha = alpha, **args)

# Numba funcs

In [30]:
import numba as nb

@nb.njit(nb.float64[:](nb.int64[:]))
def normalize(X):
  m = np.max(X)
  return (m - X) / (m - np.min(X) + 1e-12) 

@nb.njit([nb.float64[:](nb.float64[:], nb.float64, nb.float64),
          nb.float64[:](nb.float64[:], nb.int64, nb.float64),
          nb.float64[:](nb.float64[:], nb.float64, nb.int64),
         nb.float64[:](nb.float64[:], nb.int64, nb.int64)])
def normalize_to_range(X, l, r):
  m = np.min(X)
  return (X - m) / (np.max(X) - m + 1e-12) * (r -l) + l



@nb.njit([nb.types.UniTuple(nb.float64[:],2) (nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:]),
          ])
def sum_by_group(groups, values):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values = np.cumsum(values)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values = values[index]
    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    return groups, values


@nb.njit([nb.types.UniTuple(nb.float64[:], 2) (nb.float64[:], nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:], nb.int64[:]),
          ])
def sum_by_group_exclude(groups, values, exclude):
    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    
    in_group, in_value =  sum_by_group(groups[inclusive], values[inclusive])
    return in_group, in_value


@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.float64, nb.float64, nb.float64 , nb.float64, nb.int64), 
          nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.int64 , nb.int64, nb.int64), 
          ])
def _get_item_neighbours(edges, lts, fts, prev_click, prev_cart, prev_order, pagerank, alpha, click_weight, cart_weight, order_weight, day):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order

  coclick = np.where(edges[:,4] < prev_click, edges[:,4], prev_click)
  cocart = np.where(edges[:,5] < prev_click, edges[:,5], prev_click)
  coorder = np.where(edges[:,6] < prev_click, edges[:,6], prev_click)

  weight = coclick * click_weight + cocart * cart_weight  + coorder * order_weight 



  #check if union, if yes time diff = 0, else = min(fts1 - lts2, fts2 - lts1)
  is_end_later = np.where(edges[:, 2] > lts, True, False)

  is_union = np.empty(edges.shape[0], dtype = np.bool_)
  is_union[is_end_later] = np.where(edges[is_end_later, 3] < lts, True, False)
  is_end_sooner = ~is_end_later
  is_union[is_end_sooner] = np.where(edges[is_end_sooner, 2] > fts, True, False)

  time_diff = np.empty(edges.shape[0],  dtype = np.float32)
  
  time_diff[is_union] = 0

  not_union_and_later = ~is_union & is_end_later
  not_union_and_sooner = ~is_union & is_end_sooner

  time_diff[not_union_and_later] = np.abs(lts - edges[not_union_and_later, 3]) / day
  time_diff[not_union_and_sooner] = np.abs(fts - edges[not_union_and_sooner, 2]) / day
  time_decay = alpha ** time_diff
  
  fitness = weight * time_decay * pagerank
  return sum_by_group(edges[:, 1], fitness)


@nb.njit(nb.int64[:](nb.int64[:, :], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.int64, nb.float64, nb.float64, nb.float64) )
def _get_test_items_when_test_many(edges, beta, test_click_weight, test_cart_weight, test_order_weight_l, test_order_weight_s, num_count_order, test_click_count, test_cart_count, test_order_count):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder

  order = np.argsort(- (edges[:,3] * test_click_count + edges[:, 4] * test_cart_count + edges[:, 5] * test_order_count) )[:num_count_order]

  #get time + type weighted rank
  time_decay = beta ** normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, test_order_weight_l, test_order_weight_s)

  weight = edges[:, 3] * test_click_weight  + edges[:, 4] * test_cart_weight + order_weight 

  fitness =  time_decay * weight

  order1 = np.argsort(- fitness ) [:20]

  #get at least num_count_order from most click cart and then the rest from time + type weighted
  k = 0
  for o in order1:
    if o in order:
      continue
    order = np.hstack((order, np.array([o])))
    if len(order) == 20:
      break
    
  return edges[order, 1]

@nb.njit(nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:, :], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64) )
def _get_test_items(edges, beta, test_click_weight, test_cart_weight, test_order_weight_l, test_order_weight_s):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  
  time_decay = beta ** normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, test_order_weight_l, test_order_weight_s)

  weight = edges[:, 3] * test_click_weight + edges[:, 4] * test_cart_weight + order_weight 

  fitness =  time_decay * weight

  order = np.argsort(- fitness )  

  return edges[order, 1], fitness +  order_weight / 2
  

@nb.njit(nb.types.UniTuple(nb.int64[:],6)(nb.int64[:, :]))
def _get_user_neighbours(edges):
  #1 target 2 ltss 3 ftss 4 click 5 cart 6 order
  return edges[:,1], edges[:,2], edges[:,3], edges[:, 4], edges[:, 5], edges[:, 6]


@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.float64[:], nb.int64, nb.float64),
          nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.int64[:], nb.int64, nb.float64),
          ])
def find_topk_user(cousers, cofitness, min_topk_user, cofitness_thres):
  #sort cousers ascending by id
  order = np.argsort(cousers)
  cousers = cousers[order]
  cofitness = cofitness[order]

  #calculate groupby sum of cousers
  cumcofitness = np.cumsum(cofitness)

  #find unique
  index = np.ones(len(cousers), 'bool')

  #this is the last index of each group
  index[:-1] = cousers[1:] != cousers[:-1]
  cumcofitness = cumcofitness[index]
  

  #find start and end of groups
  nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))
    
  #caluculate groupby sum of cousers
  cumcofitness[1:] = cumcofitness[1:] - cumcofitness[:-1]

  #get topk user
  #dynamic top user
  total_fitness = np.sum(cumcofitness)
  order_fit = np.argsort(-cumcofitness)
  k = 0
  remain = 0
  thresh_hold = cofitness_thres * total_fitness
  while remain < thresh_hold:
    remain += cumcofitness[order_fit][k]
    k+=1 
  
  if k < min_topk_user:
    k = min_topk_user

  order_fit = order_fit[:k]

  starts = nodes[order_fit]
  ends = nodes[order_fit + 1]
  
  filter = np.zeros(len(cousers), 'bool')
  for i in range(len(starts)):
    filter[starts[i] : ends[i]] = 1
  
  return cousers[filter], cofitness[filter], order, filter



# Step

In [31]:
def get_type_items(user, clicks, carts, orders, len_clicks, len_carts, len_orders, action):

  #get item in tests

  items, fitness = get_test_items(user, BETA, action = action)
  
  if action == 0:
    lens = len_clicks
    preds = clicks
  if action == 1:
    lens = len_carts
    preds = carts
  if action == 2:
    lens = len_orders
    preds = orders



  lens.append(len(items))
  
  if len(items)  > 19:
    preds.append(items.tolist()[:20])
  
  else:
  
    num_more = 20 - len(items)
    
    
    
    #get couser
    coalls = [get_user_neighbours(items[i]) for i in range(len(items))]

    cofitness = np.hstack([np.full_like(coalls[i][0], fitness[i]) for i in range(len(items))])
    

    #find topk cousers
    cousers, cofitness, order, filter  = find_topk_user(np.hstack([all[0] for all in coalls]), cofitness, MIN_TOPK_COUSER, COFITNESS_THRES)
    coltss = np.hstack([all[1] for all in coalls])[order][filter]
    coftss = np.hstack([all[2] for all in coalls])[order][filter]
    coclick = np.hstack([all[3] for all in coalls])[order][filter]
    cocart = np.hstack([all[4] for all in coalls])[order][filter]
    coorder = np.hstack([all[5] for all in coalls])[order][filter]

    #get coitems
    coalls = [get_item_neighbours(cousers[i], coltss[i], coftss[i], coclick[i], cocart[i], coorder[i], ALPHA, use_pagerank = USE_PAGERANK, use_log = USE_LOG, action = action) for i in range(len(cousers))]

    #multiply item by fitness of user which interacted with it
    cofitness = np.hstack([all[1] * cofitness[i] for i, all in enumerate(coalls)]) 
    coitems = np.hstack([all[0] for all in coalls])
    

    
    #count covisit

    coitems, cofitness= sum_by_group_exclude(coitems, cofitness, exclude =  items)
    
    #sort by count then to fitness
    order = np.argsort( -cofitness )
        
    preds.append(items.tolist() +  coitems[order[:num_more]].tolist())

    del cofitness, coalls


In [32]:
import time

def step(user, clicks, carts, orders, len_clicks, len_carts, len_orders):

  #CLICKS
  for i in range(3):
    get_type_items(user, clicks, carts, orders, len_clicks, len_carts, len_orders, i)

    
  


# Test pipeline

In [ ]:
clicks = []
carts = []
orders = []
len_clicks = []
len_carts = []
len_orders = []
for user in tqdm(session_list[:1]):
  step(user, clicks, carts, orders, len_clicks, len_carts, len_orders)

# Multiprocessing

In [33]:
def shuffle(l):  
    perm = np.random.permutation(len(l))  
    l[:] = [l[j] for j in perm]  
    return perm


def unshuffle(L, perm):  
    
    res = [[None] * len(l) for l in L]
    for i, j in enumerate(perm):
        for k, l in enumerate(L):
          res[k][j] = l[i]
          
    for i, l in enumerate(L):
      l[:] = res[i]

In [34]:
from tqdm import tqdm
import multiprocessing

def batch_proc(proc_num, batch, return_dict):
  clicks = []
  carts = []
  orders = []
  len_clicks = []
  len_carts = []
  len_orders = []
  for user in tqdm(batch):

    step(user, clicks, carts, orders, len_clicks, len_carts, len_orders)

  return_dict[proc_num] =  clicks, carts, orders, len_clicks, len_carts, len_orders
    

In [35]:
perm = shuffle(session_list)

In [36]:
#divide into batches
NUM_CPU = multiprocessing.cpu_count()
batch_size = len(session_list) // NUM_CPU
start = np.arange(NUM_CPU) * batch_size
end = start + batch_size
end[-1] = len(session_list)
batches = [session_list[start[i] : end[i]] for i in range(len(start))]

In [37]:
manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = [multiprocessing.Process(target=batch_proc, args=(i, batch, return_dict)) for i, batch in enumerate(batches)]
len(jobs)

40

In [38]:
%%time
for job in jobs:
  job.start()
for job in jobs:
  job.join()
# return_dict[0]

100%|██████████| 41795/41795 [2:28:44<00:00,  4.68it/s]


CPU times: user 42min 17s, sys: 9min 59s, total: 52min 16s
Wall time: 2h 28min 47s


In [39]:
clicks = []
carts = []
orders = []
len_clicks = []
len_carts = []
len_orders = []
from tqdm.notebook import tqdm
for i in tqdm(range(len(return_dict))):
    clicks.extend(return_dict[i][0])
    carts.extend(return_dict[i][1])
    orders.extend(return_dict[i][2])
    len_clicks.extend(return_dict[i][3])
    len_carts.extend(return_dict[i][4])
    len_orders.extend(return_dict[i][5])


  0%|          | 0/40 [00:00<?, ?it/s]

In [40]:
del return_dict

In [41]:
unshuffle([clicks,carts,orders,len_clicks,len_carts,len_orders, session_list], perm)


# Create submission

In [42]:
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(2241, 18576, 5015409)

In [43]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in clicks] if JOIN else [[int(j) - 100000000 for j in i] for i in clicks]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in carts] if JOIN else [[int(j) - 100000000 for j in i] for i in carts]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in orders] if JOIN else [[int(j) - 100000000 for j in i] for i in orders]

In [44]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [45]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [46]:
submission1

,session_type,labels
0,12899779_clicks,59625 737445 523135 448688 637538 1790770 7316...
1,12899780_clicks,1142000 736515 973453 582732 1758603 487136 88...
2,12899781_clicks,199008 918667 194067 57315 141736 1460571 8249...
3,12899782_clicks,779477 1494780 476063 889671 975116 562753 834...
4,12899783_clicks,1817895 300127 255297 607638 1754419 1216820 1...
...,...,...
1671798,14571577_clicks,1141710 1276792 1004292 842555 1666114 86916 6...
1671799,14571578_clicks,519105 977826 1671592 815460 1811714 822641 15...
1671800,14571579_clicks,739876 1209992 1750859 785544 1550479 51363 77...
1671801,14571580_clicks,202353 433425 871658 1314576 1231403 925638 68...


In [47]:
final = pd.concat([submission1, submission2, submission3])

In [48]:
del submission1, submission2, submission3

In [49]:
final

,session_type,labels
0,12899779_clicks,59625 737445 523135 448688 637538 1790770 7316...
1,12899780_clicks,1142000 736515 973453 582732 1758603 487136 88...
2,12899781_clicks,199008 918667 194067 57315 141736 1460571 8249...
3,12899782_clicks,779477 1494780 476063 889671 975116 562753 834...
4,12899783_clicks,1817895 300127 255297 607638 1754419 1216820 1...
...,...,...
1671798,14571577_orders,1141710 1276792 1004292 842555 86916 631085 13...
1671799,14571578_orders,519105 977826 1671592 815460 822641 1811714 15...
1671800,14571579_orders,739876 1209992 1750859 1550479 785544 770418 5...
1671801,14571580_orders,202353 433425 925638 871658 1314576 1231403 68...


# Eval valid

In [ ]:
import pandas as pd

def get_score(df, split = False):
    submission = df.copy()
    test_labels = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/điệp viên/splitted_data/test_labels.parquet')
    test_labels = test_labels[test_labels['session'] <= np.amax(session_list)]
    
    submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
    
    session = set(test_labels['session'].unique().tolist())
    for j in session_list:
      assert j in session


    submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
    
    if split:
      submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

    del submission

    recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

    print (f"Score : {(recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()}")
    return recall_per_type

In [ ]:
get_score(final, JOIN)

Score : 0.5050865349572584


type
carts     0.350457
clicks    0.479013
orders    0.586747
dtype: float64

# Save sumission

In [50]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_4_submission.csv', index = False)

#Submit


In [51]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle.json ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle/kaggle.json': No such file or directory


In [53]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_9_4_submission.csv -m "new covisitation"

100% 783M/783M [00:19<00:00, 42.2MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System

#Submit 1

In [65]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle1.json ~/.kaggle
mv ~/.kaggle/kaggle1.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [67]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_9_2_submission.csv -m "validate mega graph tool test sampling"

100% 174M/174M [00:04<00:00, 41.4MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System

# Create submission test only 

In [56]:
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(2241, 18576, 5015409)

In [57]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_clicks[m]]]) for m, a in enumerate(clicks)] if JOIN else [[int(j) - 100000000 for j in i[ : len_clicks[m]]] for m, i in enumerate(clicks)]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_carts[m]]]) for m, a in enumerate(carts)] if JOIN else [[int(j) - 100000000 for j in i[ : len_carts[m]]] for m, i in enumerate(carts)]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_orders[m]]]) for m, a in enumerate (orders)] if JOIN else [[int(j) - 100000000 for j in i[ : len_orders[m]]] for m, i in enumerate(orders)]

In [58]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [59]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [60]:
submission1

,session_type,labels
0,12899779_clicks,59625
1,12899780_clicks,1142000 736515 973453 582732
2,12899781_clicks,199008 918667 194067 57315 141736
3,12899782_clicks,779477 1494780 476063 889671 975116 562753 834...
4,12899783_clicks,1817895 300127 255297 607638 1754419 1216820 1...
...,...,...
1671798,14571577_clicks,1141710
1671799,14571578_clicks,519105
1671800,14571579_clicks,739876
1671801,14571580_clicks,202353


In [61]:
final = pd.concat([submission1, submission2, submission3])

In [62]:
del submission1, submission2, submission3

In [63]:
final

,session_type,labels
0,12899779_clicks,59625
1,12899780_clicks,1142000 736515 973453 582732
2,12899781_clicks,199008 918667 194067 57315 141736
3,12899782_clicks,779477 1494780 476063 889671 975116 562753 834...
4,12899783_clicks,1817895 300127 255297 607638 1754419 1216820 1...
...,...,...
1671798,14571577_orders,1141710
1671799,14571578_orders,519105
1671800,14571579_orders,739876
1671801,14571580_orders,202353


# Save sumission

In [64]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_5_submission.csv', index = False)